# Quantum Teleportation

Quantum teleportation is a protocol that allows the *state* of a qubit to be transferred from one location (Alice) to another (Bob), without physically moving the qubit itself.  This is achieved using entanglement and classical communication.  It's important to note that teleportation *doesn't* transmit information faster than light, and it *doesn't* copy the qubit's state (which would violate the no-cloning theorem). Instead, it *transfers* the state, destroying the original qubit's state in the process.

**Key Concepts:**

*   **Entanglement:** A Bell pair (a maximally entangled state of two qubits) is shared between Alice and Bob.
*   **Classical Communication:** Alice performs measurements on her qubits and sends the *classical* results (two bits) to Bob.
*   **Conditional Gates:** Bob uses the classical information from Alice to apply specific gates to his qubit, reconstructing the original state.

**We will:**

1.  Implement the quantum teleportation protocol.
2.  Explain each step and the role of each gate.
3.  Provide an example, teleporting a specific qubit state.


## Step 1: Import Necessary Libraries

We'll use Qiskit.

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
import numpy as np

We begin by importing the necessary libraries:

*   **Qiskit:**
    *   `QuantumCircuit`: For building quantum circuits.
    *   `QuantumRegister`: For creating groups of qubits.
    *   `ClassicalRegister`: For storing measurement results.
    *   `transpile`: For optimizing circuits for a specific backend.
*    **Qiskit Aer:**
     * `AerSimulator`: For simulating circuits.

## Step 2: Define Helper Functions

We'll define several helper functions to make the main teleportation function more readable and modular.

In [2]:
def create_bell_pair(qc, a, b):
    qc.h(a) # Apply Hadamard to the first qubit
    qc.cx(a, b) # Apply CNOT, entangling a and b


def alice_gates(qc, psi, a):
    qc.cx(psi, a) # CNOT with psi as control, a as target
    qc.h(psi) # Hadamard to the psi qubit


def measure_and_send(qc, a, b, c_alice, c_bob):
    qc.measure(a, c_alice[0]) # Measure qubit a, store in classical bit 0
    qc.measure(b, c_alice[1])  # Measure qubit b, store in classical bit 1


def bob_gates(qc, qubit, c_alice, c_bob):
     # Apply gates if classical bits are 1 (use c_if)
    with qc.if_test((c_alice[1], 1)):
        qc.x(qubit)
    with qc.if_test((c_alice[0], 1)):
        qc.z(qubit)

*   **`create_bell_pair(qc, a, b)`:**
    *   Creates a Bell pair (maximally entangled state) between qubits `a` and `b` of the quantum circuit `qc`.
    *   `qc.h(a)`: Applies a Hadamard gate to qubit `a`, putting it in superposition.
    *   `qc.cx(a, b)`: Applies a CNOT gate with `a` as control and `b` as target, entangling the two qubits.

*   **`alice_gates(qc, psi, a)`:**
    *   Applies the gates Alice performs on her qubits (`psi` and `a`) in the quantum circuit `qc`.
    *   `qc.cx(psi, a)`: Applies a CNOT gate with `psi` (the qubit to be teleported) as control and `a` (Alice's Bell pair qubit) as target.
    *   `qc.h(psi)`: Applies a Hadamard gate to `psi`.

*   **`measure_and_send(qc, a, b, c_alice, c_bob)`:**
    *   Performs measurements on Alice's qubits (`a` and `b`) in the quantum circuit `qc` and stores the results in classical registers.
    *   `qc.measure(a, c_alice[0])`: Measures qubit `a` and stores the result in the first bit of the `c_alice` classical register.
    *   `qc.measure(b, c_alice[1])`: Measures qubit `b` and stores the result in the second bit of the `c_alice` classical register.

*   **`bob_gates(qc, qubit, c_alice, c_bob)`:**
    *   Applies Bob's correction gates to his qubit (`qubit`) in the quantum circuit `qc`, based on the classical bits received from Alice (`c_alice`).
    *    `with qc.if_test((c_alice[1], 1)): qc.x(qubit)`:  If the *second* bit of `c_alice` is 1, apply an X gate to Bob's qubit. This is a *conditional* gate.
    *    `with qc.if_test((c_alice[0], 1)): qc.z(qubit)`: If the *first* bit of `c_alice` is 1, apply a Z gate to Bob's qubit. This is a *conditional* gate.

These helper functions break down the teleportation protocol into logical steps, improving code readability and organization.

## Step 3: Define the Main Teleportation Function

This function orchestrates the entire teleportation process.

In [3]:
def quantum_teleportation(initial_state_prep = None):

    # Create registers
    q = QuantumRegister(3, 'q')  # Three qubits: psi, a, b
    c_alice = ClassicalRegister(2, 'c_alice')  # Two classical bits for Alice
    c_bob = ClassicalRegister(1, 'c_bob')     # One classical bit for Bob
    qc = QuantumCircuit(q, c_alice, c_bob)


    # If an initial_state_prep circuit is provided, apply it to qubit 0.
    if initial_state_prep is not None:
        qc.compose(initial_state_prep, [0], inplace=True)


    # Step 1: Create a Bell pair between Alice and Bob
    create_bell_pair(qc, 1, 2)  # Entangle qubits a (q[1]) and b (q[2])
    qc.barrier()

    # Step 2: Alice interacts psi (q[0]) with her Bell pair qubit (a)
    alice_gates(qc, 0, 1)
    qc.barrier()

    # Step 3: Alice measures her two qubits and sends the results to Bob
    measure_and_send(qc, 0, 1, c_alice, c_bob)
    qc.barrier()
    # Step 4: Bob applies gates based on Alice's measurement results
    bob_gates(qc, 2, c_alice, c_bob)

    # Measure Bob's qubit – this is just for verification, not part of teleportation itself.
    qc.measure(2, c_bob[0])


    return qc

The `quantum_teleportation` function implements the complete protocol:

*   **Function Definition:** `def quantum_teleportation(initial_state_prep=None):`
    *   `initial_state_prep`:  An *optional* `QuantumCircuit` object. If provided, this circuit will be used to prepare the initial state of the qubit to be teleported (qubit 0).  If `None`, qubit 0 starts in the default |0⟩ state. This allows us to teleport *any* arbitrary single-qubit state.

*   **Register Creation:**
    *   `q = QuantumRegister(3, 'q')`: Creates a quantum register named `q` with three qubits:
        *   `q[0]`: `psi` - The qubit whose state will be teleported.
        *   `q[1]`: `a` - Alice's half of the Bell pair.
        *   `q[2]`: `b` - Bob's half of the Bell pair.
    *   `c_alice = ClassicalRegister(2, 'c_alice')`: Creates a classical register for Alice's two measurement results.
    *   `c_bob = ClassicalRegister(1, 'c_bob')`: Creates a classical register for Bob's measurement (used for *verification* in this example, not strictly part of the teleportation protocol).
    *   `qc = QuantumCircuit(q, c_alice, c_bob)`: Creates the main quantum circuit.

* **Initial State Preparation (Optional):**
   * `if initial_state_prep is not None:`: Checks if an initial state preparation circuit was provided
   * `qc.compose(initial_state_prep, [0], inplace=True)`:  If provided, *prepends* the `initial_state_prep` circuit to the beginning of the `qc` circuit, applying it only to qubit 0 (`[0]`).  This sets up the state we want to teleport.

*   **Step 1: Create Bell Pair:**
    *   `create_bell_pair(qc, 1, 2)`: Calls the helper function to create a Bell pair between Alice's qubit (`q[1]`) and Bob's qubit (`q[2]`).
    * `qc.barrier()`: Adds a barrier for visual clarity in the circuit diagram.  It doesn't affect the computation.

*   **Step 2: Alice's Gates:**
    *   `alice_gates(qc, 0, 1)`: Calls the helper function to apply Alice's gates to her qubits (`q[0]` and `q[1]`).
    *   `qc.barrier()`: Adds another barrier.

*   **Step 3: Measure and Send:**
    *   `measure_and_send(qc, 0, 1, c_alice, c_bob)`: Calls the helper function to measure Alice's qubits and store the results in `c_alice`.
     * `qc.barrier()`: Adds another barrier.

*   **Step 4: Bob's Gates:**
    *   `bob_gates(qc, 2, c_alice, c_bob)`: Calls the helper function to apply Bob's correction gates to his qubit (`q[2]`) based on Alice's measurement results.

*   **Verification Measurement:**
    *    `qc.measure(2, c_bob[0])`: Measures Bob's qubit (`q[2]`) and stores the result in `c_bob[0]`. This is *not* strictly part of the teleportation protocol itself. It's included here to *verify* that Bob's qubit has the same state as the original `psi` qubit.
*   **Return Value:** `return qc`: Returns the complete quantum circuit.

**The Protocol (in words):**
1. **Setup:** Alice and Bob share an entangled Bell pair. Alice also has the qubit (`psi`) she wants to teleport.
2. **Interaction:** Alice interacts `psi` with her half of the Bell pair (using CNOT and Hadamard gates).
3. **Measurement:** Alice measures both of her qubits and obtains two classical bits of information.
4. **Communication:** Alice sends these two classical bits to Bob (using any classical communication channel).
5. **Reconstruction:** Bob, based on the two bits he received, applies either an X gate, a Z gate, both, or neither to his half of the Bell pair.  This reconstructs the original state of `psi` on Bob's qubit.


## Step 4: Example Usage

Let's demonstrate teleporting the |+⟩ state and the |1⟩ state. We also show how to prepare an arbitrary initial state.

In [8]:
# --- Example Usage 1: Teleporting |+> state---
plus_prep = QuantumCircuit(1)
plus_prep.h(0)

teleportation_circuit = quantum_teleportation(plus_prep)


# Simulate the circuit
simulator = AerSimulator()
compiled_circuit = transpile(teleportation_circuit, simulator)
job = simulator.run(compiled_circuit, shots=1024)
result = job.result()
counts = result.get_counts(teleportation_circuit)

print("Counts:", counts)
print(teleportation_circuit.draw())

# --- Example Usage 2: Teleporting |1> state---
one_prep = QuantumCircuit(1)
one_prep.x(0)

teleportation_circuit_one = quantum_teleportation(one_prep)

simulator = AerSimulator()
compiled_circuit_one = transpile(teleportation_circuit_one, simulator)
job_one = simulator.run(compiled_circuit_one, shots=1024)
result_one = job_one.result()
counts_one = result_one.get_counts(teleportation_circuit_one)

print("\\nCounts for |1>:", counts_one)
print(teleportation_circuit_one.draw())


# --- Example Usage 3: Teleporting a superposition ---
# Create a circuit to prepare a superposition state:  1/sqrt(2) |0> + i/sqrt(2) |1>
superposition_prep = QuantumCircuit(1)
superposition_prep.h(0)
superposition_prep.s(0)  # Apply S gate to introduce the 'i' phase

teleportation_circuit_superpos = quantum_teleportation(superposition_prep)

# Simulate
simulator = AerSimulator()
compiled_circuit_sp = transpile(teleportation_circuit_superpos, simulator)
job_sp = simulator.run(compiled_circuit_sp, shots=1024)
result_sp = job_sp.result()
counts_sp = result_sp.get_counts(teleportation_circuit_superpos)

print("\\nCounts for superposition:", counts_sp)
print(teleportation_circuit_superpos.draw())

Counts: {'0 01': 120, '1 10': 128, '0 11': 130, '1 11': 124, '1 00': 142, '0 10': 140, '0 00': 131, '1 01': 109}
           ┌───┐      ░      ┌───┐ ░ ┌─┐    ░                                 »
      q_0: ┤ H ├──────░───■──┤ H ├─░─┤M├────░─────────────────────────────────»
           ├───┤      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐ ░                                 »
      q_1: ┤ H ├──■───░─┤ X ├──────░──╫─┤M├─░─────────────────────────────────»
           └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ░      ┌──────     ┌───┐ ───────┐ »
      q_2: ─────┤ X ├─░────────────░──╫──╫──░──────┤ If-0  ────┤ X ├  End-0 ├─»
                └───┘ ░            ░  ║  ║  ░      └──╥───     └───┘ ───────┘ »
                                      ║  ║    ┌───────╨───────┐               »
c_alice: 2/═══════════════════════════╩══╩════╡ c_alice_1=0x1 ╞═══════════════»
                                      0  1    └───────────────┘               »
  c_bob: 1/═══════════════════════════════════════════════════════════════════»
       

*   **Example 1: Teleporting |+>:**
    *   `plus_prep = QuantumCircuit(1)`: Creates a circuit to prepare the |+⟩ state.
    *   `plus_prep.h(0)`: Applies a Hadamard gate to qubit 0, creating the |+⟩ state (1/√2 |0⟩ + 1/√2 |1⟩).
    *   `teleportation_circuit = quantum_teleportation(plus_prep)`: Calls `quantum_teleportation`, passing `plus_prep` to initialize qubit 0 to |+⟩.
*   `simulator = ...`: Sets up the simulator, transpiles, runs the simulation, and gets the results, similar to previous examples.
*   **Example 2: Teleporting |1>:**
    *   `one_prep = QuantumCircuit(1)`: Creates a circuit to prepare the |1⟩ state.
    *   `one_prep.x(0)`: Applies an X gate to qubit 0, creating the |1⟩ state.
    *   `teleportation_circuit_one = quantum_teleportation(one_prep)`: Calls `quantum_teleportation` with `one_prep`.
*   The rest is analogous to Example 1.
*  **Example 3: Teleporting a superposition:**
     *  `superposition_prep = QuantumCircuit(1)`: Create a Quantum Circuit
     * `superposition_prep.h(0)`: Creates an equal super position
     * `superposition_prep.s(0)`: Apply S gate to introduce the 'i' phase
    *   `teleportation_circuit_superpos = quantum_teleportation(superposition_prep)`: Calls `quantum_teleportation` with `superposition_prep`.
* **Simulation and Output:** The code simulates the circuit and prints the measurement counts.  Because of the probabilistic nature of quantum measurement (even though teleportation itself is deterministic), we run the simulation many times (`shots=1024`) to get good statistics. We expect to see Bob's qubit (measured into `c_bob`) in the same state as the original `psi` qubit. The counts will reflect the probabilities associated with the initial state.
* **Drawing:** In all examples the circuit's drawing is printed.